In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

tqdm.pandas()

from geopy import distance

## Data from zenodo archive

https://zenodo.org/record/7923702

In [2]:
january = pd.read_csv(
    "D:/a.salgas/Documents/opensky_2019/flightlist_20190101_20190131.csv",
    dtype={1: "string"},
)
february = pd.read_csv(
    "D:/a.salgas/Documents/opensky_2019/flightlist_20190201_20190228.csv",
    dtype={1: "string"},
)
march = pd.read_csv(
    "D:/a.salgas/Documents/opensky_2019/flightlist_20190301_20190331.csv",
    dtype={1: "string"},
)
april = pd.read_csv(
    "D:/a.salgas/Documents/opensky_2019/flightlist_20190401_20190430.csv",
    dtype={1: "string"},
)
may = pd.read_csv(
    "D:/a.salgas/Documents/opensky_2019/flightlist_20190501_20190531.csv",
    dtype={1: "string"},
)
june = pd.read_csv(
    "D:/a.salgas/Documents/opensky_2019/flightlist_20190601_20190630.csv",
    dtype={1: "string"},
)
july = pd.read_csv(
    "D:/a.salgas/Documents/opensky_2019/flightlist_20190701_20190731.csv",
    dtype={1: "string"},
)
august = pd.read_csv(
    "D:/a.salgas/Documents/opensky_2019/flightlist_20190801_20190831.csv",
    dtype={1: "string"},
)
september = pd.read_csv(
    "D:/a.salgas/Documents/opensky_2019/flightlist_20190901_20190930.csv",
    dtype={1: "string"},
)
october = pd.read_csv(
    "D:/a.salgas/Documents/opensky_2019/flightlist_20191001_20191031.csv",
    dtype={1: "string"},
)
november = pd.read_csv(
    "D:/a.salgas/Documents/opensky_2019/flightlist_20191101_20191130.csv",
    dtype={1: "string"},
)
december = pd.read_csv(
    "D:/a.salgas/Documents/opensky_2019/flightlist_20191201_20191231.csv",
    dtype={1: "string"},
)


opensky_extract = pd.concat(
    [
        january,
        february,
        march,
        april,
        may,
        june,
        july,
        august,
        september,
        october,
        november,
        december,
    ],
    axis=0,
).reset_index()

In [16]:
opensky_2019 = opensky_extract.copy()

In [17]:
keep_time = False

# selecting only the columns interesting us
opensky_2019 = opensky_2019.loc[:, ["day", "destination", "origin", "icao24"]]
# Drop flight with no known origin and destination (mandatory because of our computation method)
size_before_drops = len(opensky_2019.index)
# opensky_2019.dropna(subset=["destination", "origin"], inplace=True)
size_after_drops = len(opensky_2019.index)
print(
    "{}% of flights deleted after removing "
    "flights with no origin and destination. {} Flights in the dataset".format(
        (size_before_drops - size_after_drops) / size_before_drops * 100,
        size_after_drops,
    )
)

# Opensky only gets aircraft transponder code, not its registration.
# Hopefully, it can be merged with an aircraft database, for example opensky aircraft database

ac_ref = pd.read_csv(
    "../01_to_03_bis_opensky_only/data/aircraft-database-complete-2022-05.csv", sep=";"
)


# Even if each aircraft has a unique icao24 code, some are present several times in the database.
# Duplicates are therefore dropped before merging the dataframes.
ac_ref = ac_ref[["icao24", "typecode", "operator_os", "operatoricao"]].drop_duplicates(
    subset="icao24", keep="last"
)
opensky_2019 = pd.merge(
    opensky_2019, ac_ref, left_on="icao24", right_on="icao24", how="left", sort=False
)

# we use ailine ICAO code, but it is named IATA in the following for compatibility reasons
opensky_2019.rename(
    columns={"operatoricao": "airline_iata", "typecode": "aircraft_type"}, inplace=True
)
# opensky_2019.aircraft_type = opensky_2019.aircraft_type.replace("ZZZZ", "zzz")
opensky_2019.aircraft_type.fillna("Unkwown", inplace=True)
opensky_2019.airline_iata.fillna("Unkwown", inplace=True)

# # Counting flight per aircraft type /route/airline each month
if keep_time:
    # Converting dates to a suitable format
    opensky_2019["year"] = opensky_2019.apply(
        lambda x: get_month_year(x["day"])[0], axis=1
    )
    opensky_2019["month"] = opensky_2019.apply(
        lambda x: get_month_year(x["day"])[1], axis=1
    )

    opensky_2019 = opensky_2019.groupby(
        ["airline_iata", "origin", "destination", "aircraft_type", "month", "year"],
        as_index=False,
        dropna=False,
    ).size()
else:
    # allows to significantly reduce the size of the data by eliminating the temporal variables
    #         by aggregating whole dataframe values
    opensky_2019 = opensky_2019.groupby(
        ["airline_iata", "origin", "destination", "aircraft_type"],
        as_index=False,
        dropna=False,
    ).size()

opensky_2019.rename(columns={"size": "n_flights"}, inplace=True)
print(
    "Size of df after_grouping: {}; number of flights: {}".format(
        len(opensky_2019.index), opensky_2019.n_flights.sum()
    )
)


ac_cla = pd.read_csv(
    "../03_routes_schedule/data/open_sky/aircraft_classification.csv", sep=";"
)
opensky_2019 = opensky_2019.merge(
    ac_cla, left_on="aircraft_type", right_on="aircraft_osky", how="left"
)
opensky_2019.drop(columns=["aircraft_osky"], inplace=True)

opensky_2019.acft_icao.fillna("Unknown", inplace=True)
opensky_2019.acft_class.fillna("Unknown", inplace=True)
opensky_2019.seymour_proxy.fillna("zzz", inplace=True)

print(
    "Size of df after ac info: {}; number of flights {}".format(
        len(opensky_2019.index), opensky_2019.n_flights.sum()
    )
)

# similarly, we merge an airport database to have info on airport iata designator, and gps coordinates.
# All major airports have IATA designator.

arpt_ref = pd.read_csv(
    "../03_routes_schedule/data/ourairports.csv",
    sep=";",
    keep_default_na=False,
    na_values="",
)

opensky_2019 = opensky_2019.merge(
    arpt_ref[
        [
            "ident",
            "iata_code",
            "longitude_deg",
            "latitude_deg",
            "iso_country",
            "continent",
        ]
    ],
    left_on="origin",
    right_on="ident",
    how="left",
    sort=False,
)
opensky_2019.rename(
    columns={
        "longitude_deg": "origin_lon",
        "latitude_deg": "origin_lat",
        "iso_country": "origin_country",
        "continent": "origin_continent",
        "iata_code": "origin_iata",
    },
    inplace=True,
)

# opensky_2019.dropna(subset=["origin_iata"], inplace=True)

opensky_2019[
    ["origin","origin_lon", "origin_lat", "origin_country", "origin_continent", "origin_iata"]
] = opensky_2019[
    ["origin","origin_lon", "origin_lat", "origin_country", "origin_continent", "origin_iata"]
].fillna(
    "Unknown"
)

opensky_2019 = opensky_2019.merge(
    arpt_ref[
        [
            "ident",
            "iata_code",
            "longitude_deg",
            "latitude_deg",
            "iso_country",
            "continent",
        ]
    ],
    left_on="destination",
    right_on="ident",
    how="left",
    sort=False,
)
opensky_2019.rename(
    columns={
        "destination": "dest",
        "longitude_deg": "dest_lon",
        "latitude_deg": "dest_lat",
        "iso_country": "dest_country",
        "continent": "dest_continent",
        "iata_code": "dest_iata",
    },
    inplace=True,
)

# opensky_2019.dropna(subset=["dest_iata"], inplace=True)

opensky_2019[
    ["dest","dest_lon", "dest_lat", "dest_country", "dest_continent", "dest_iata"]
] = opensky_2019[
    ["dest","dest_lon", "dest_lat", "dest_country", "dest_continent", "dest_iata"]
].fillna(
    "Unknown"
)

opensky_2019.drop(columns=["ident_x", "ident_y"], inplace=True)


opensky_2019.dropna(
    subset=["dest_lon", "origin_lon", "dest_lat", "origin_lat"], inplace=True
)

print(
    "Size of df after arpt info: {}, number of flights: {}".format(
        len(opensky_2019.index), opensky_2019.n_flights.sum()
    )
)

0.0% of flights deleted after removing flights with no origin and destination. 31218164 Flights in the dataset


C:\Users\a.salgas.ISAE-SUPAERO\AppData\Local\Temp\ipykernel_17484\3311929180.py:20: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  ac_ref = pd.read_csv(


Size of df after_grouping: 2150912; number of flights: 31218164
Size of df after ac info: 2150912; number of flights 31218164
Size of df after arpt info: 2150912, number of flights: 31218164


In [18]:
os_df = opensky_2019.copy()

os_df

,airline_iata,origin,dest,aircraft_type,n_flights,acft_icao,acft_class,seymour_proxy,origin_iata,origin_lon,origin_lat,origin_country,origin_continent,dest_iata,dest_lon,dest_lat,dest_country,dest_continent
0,AAB,BIRK,Unknown,CL35,1,CL35,PJ,zzz,RKV,-21.940599,64.129997,IS,EU,Unknown,Unknown,Unknown,Unknown,Unknown
1,AAB,EBAW,EBMB,CL35,1,CL35,PJ,zzz,ANR,4.46028,51.1894,BE,EU,Unknown,4.49,50.913,BE,EU
2,AAB,EBAW,EDDN,CL35,1,CL35,PJ,zzz,ANR,4.46028,51.1894,BE,EU,NUE,11.078056,49.498699,DE,EU
3,AAB,EBAW,EGLF,CL35,1,CL35,PJ,zzz,ANR,4.46028,51.1894,BE,EU,FAB,-0.776333,51.275799,GB,EU
4,AAB,EBAW,Unknown,CL35,1,CL35,PJ,zzz,ANR,4.46028,51.1894,BE,EU,Unknown,Unknown,Unknown,Unknown,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2150907,bfx,Unknown,LIMN,PRM1,1,PRM1,PJ,zzz,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,8.66922,45.529598,IT,EU
2150908,bfx,Unknown,LOWW,PRM1,1,PRM1,PJ,zzz,Unknown,Unknown,Unknown,Unknown,Unknown,VIE,16.5697,48.110298,AT,EU
2150909,bfx,Unknown,LSZH,PRM1,1,PRM1,PJ,zzz,Unknown,Unknown,Unknown,Unknown,Unknown,ZRH,8.548056,47.458056,CH,EU
2150910,bfx,Unknown,UKKK,PRM1,1,PRM1,PJ,zzz,Unknown,Unknown,Unknown,Unknown,Unknown,IEV,30.45194,50.40194,UA,EU


In [19]:
fleet = pd.read_excel("data/planespotters_fleet.xlsx")
fleet["Seat Total"] = fleet["Seat Total"].replace(0, np.nan)
fleet.dropna(subset=["Seat Total"], inplace=True)

# Adding the name/icao code match table
match = pd.read_csv("data/fleet_match.csv", sep=";")
fleet = fleet.merge(
    match, left_on="Aircraft Type", right_on="spotter_name", how="right"
).drop(columns="spotter_name")

aircraft_data = (
    fleet.groupby(["Aircraft Type", "ICAO_AC"])["Seat Total"]
    .mean()
    .reset_index()
    .drop(columns="Aircraft Type")
)

In [20]:
aircraft_data.head()

,ICAO_AC,Seat Total
0,AT43,46.897119
1,AT45,46.897119
2,AT72,70.137580
3,AT75,70.137580
4,AT76,70.137580


In [21]:
os_df = os_df.merge(aircraft_data, left_on="acft_icao", right_on="ICAO_AC", how="left")
os_df["Seat Total"] = os_df["n_flights"] * os_df["Seat Total"]
os_df = (
    os_df.groupby(
        [
            "origin",
            "dest",
            "origin_iata",
            "origin_lon",
            "origin_lat",
            "origin_country",
            "origin_continent",
            "dest_iata",
            "dest_lon",
            "dest_lat",
            "dest_country",
            "dest_continent",
            "airline_iata",
            "acft_icao",
            "acft_class",
            "seymour_proxy",
        ],
        dropna=False,
    )[["Seat Total", "n_flights"]]
    .sum()
    .reset_index()
    .rename(columns={"Seat Total": "seats"})
)

In [22]:
os_df.n_flights.sum()

31218164

In [23]:
column_mapping = {
    "origin_iata": "iata_departure",
    "dest_iata": "iata_arrival",
    "origin_lon": "departure_lon",
    "origin_lat": "departure_lat",
    "origin_country": "departure_country",
    "origin_continent": "departure_continent",
    "dest_lon": "arrival_lon",
    "dest_lat": "arrival_lat",
    "dest_country": "arrival_country",
    "dest_continent": "arrival_continent",
}


os_df.rename(columns=column_mapping, inplace=True)
os_df["source"] = "OpenSky"

In [24]:
iata_icao_convert = pd.read_csv("data/iata_icao_airline.csv", sep=";")
iata_icao_convert = iata_icao_convert.sort_values(
    by=["IATA"], ascending=False
).drop_duplicates(subset="ICAO", keep="first")

In [25]:
os_df = os_df.merge(
    iata_icao_convert[["IATA", "ICAO"]],
    left_on="airline_iata",
    right_on="ICAO",
    how="left",
)

In [26]:
os_df["IATA"] = os_df["IATA"].fillna(os_df["airline_iata"])
os_df = os_df.drop(columns=["airline_iata", "ICAO"]).rename(
    columns={"IATA": "airline_iata"}
)

In [28]:
os_df.loc[:, "distance_km"] = os_df.progress_apply(
    lambda x: distance.distance(
        (float(x.departure_lat), float(x.departure_lon)),
        (float(x.arrival_lat), float(x.arrival_lon)),
    ).km
    if not (
        pd.isna(x.departure_lat)
        or pd.isna(x.departure_lon)
        or (x.departure_lat == "Unknown")
        or (x.departure_lon == "Unknown")
        or pd.isna(x.arrival_lon)
        or pd.isna(x.arrival_lat)
        or (x.arrival_lat == "Unknown")
        or (x.arrival_lon == "Unknown")
    )
    else 0,
    axis=1,
)

  0%|          | 0/2132700 [00:00<?, ?it/s]

In [29]:
os_df
os_df["ask"] = os_df["distance_km"] * os_df["seats"]
os_df["rpk"] = os_df["ask"] * 0.824

Load factor source: https://www.icao.int/annual-report-2019/Pages/the-world-of-air-transport-in-2019.aspx

In [30]:
os_df["seats"].sum()

3666474466.4788847

## Fuel burn computation process

Problem: we do not have a seymour proxi for all the aircraft types neither do we have the seats associated to each aircrfat like in AeroSCOPE core usage. 
One could extend the proxi list, but it is not without increasing the risk of errors.  
Helicopters are particularly often in this case as shown below, followed py piston private and private jets and few turbo props.  
Fortunately, this is not the majority of the flights.

If the interested reader is willing to increase the proxy list for a particular use case, he is free to do so ;) 

In [31]:
os_df[os_df.seymour_proxy == "zzz"].groupby("acft_class")["n_flights"].sum().nlargest(
    50
)

acft_class
Unknown    5761709
HE          651767
TP          220904
PP          179375
PJ          148189
OTHER        49311
EL              19
Name: n_flights, dtype: int64

In [32]:
os_df[os_df.seymour_proxy != "zzz"].groupby("acft_class")["n_flights"].sum().nlargest(
    50
)

acft_class
NB       15104321
WB        2958229
RJ        2943718
TP        1589327
PJ        1362569
PP         214642
OTHER       34084
Name: n_flights, dtype: int64

Now, let's compute the fuel burn using the surrogate.

In [33]:
fuel_surrogate = pd.read_csv("data/FuelSurrogate.csv", sep=";")
fuel_surrogate_dict = (
    fuel_surrogate[
        ["ac_code_icao", "reduced_fuel_intercept", "reduced_fuel_a1", "reduced_fuel_a2"]
    ]
    .set_index("ac_code_icao")
    .transpose()
    .to_dict()
)
ac_list = fuel_surrogate["ac_code_icao"].unique()


def compute_fuel_seymour(fuel_surrogate_dict, icao_type, distance):
    if icao_type in ac_list and distance > 1:
        coefficients = fuel_surrogate_dict[icao_type]
        fuel_burn = (
            coefficients["reduced_fuel_intercept"]
            + coefficients["reduced_fuel_a2"] * distance
            + coefficients["reduced_fuel_a1"] * distance**2
        )
    elif distance < 1:
        fuel_burn = 0
    else:
        fuel_burn = np.nan
    return float(fuel_burn)


os_df.loc[:, "fuel_burn_seymour"] = os_df.progress_apply(
    lambda x: compute_fuel_seymour(
        fuel_surrogate_dict, x["seymour_proxy"], x["distance_km"]
    ),
    axis=1,
)

  0%|          | 0/2132700 [00:00<?, ?it/s]

In [34]:
os_df

,origin,dest,iata_departure,departure_lon,departure_lat,departure_country,departure_continent,iata_arrival,arrival_lon,arrival_lat,...,acft_class,seymour_proxy,seats,n_flights,source,airline_iata,distance_km,ask,rpk,fuel_burn_seymour
0,00AK,00AK,Unknown,-151.692524,59.947733,US,NA,Unknown,-151.692524,59.947733,...,OTHER,B747,0.000000,1,OpenSky,AIO,0.000000,0.000000e+00,0.000000,0.000000
1,00AK,00AK,Unknown,-151.692524,59.947733,US,NA,Unknown,-151.692524,59.947733,...,HE,zzz,0.000000,1,OpenSky,Unkwown,0.000000,0.000000e+00,0.000000,0.000000
2,00AK,00AK,Unknown,-151.692524,59.947733,US,NA,Unknown,-151.692524,59.947733,...,Unknown,zzz,0.000000,1,OpenSky,Unkwown,0.000000,0.000000e+00,0.000000,0.000000
3,00AK,0AK7,Unknown,-151.692524,59.947733,US,NA,Unknown,-150.963285,59.776005,...,Unknown,zzz,0.000000,3,OpenSky,Unkwown,45.118036,0.000000e+00,0.000000,NaN
4,00AK,3AK4,Unknown,-151.692524,59.947733,US,NA,Unknown,-151.337657,60.632522,...,Unknown,zzz,0.000000,2,OpenSky,Unkwown,78.781228,0.000000e+00,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2132695,ZYHE,Unknown,HEK,127.308884,50.171621,CN,AS,Unknown,Unknown,Unknown,...,WB,B763,238.988201,1,OpenSky,Unkwown,0.000000,0.000000e+00,0.000000,0.000000
2132696,ZYHE,Unknown,HEK,127.308884,50.171621,CN,AS,Unknown,Unknown,Unknown,...,Unknown,zzz,0.000000,1,OpenSky,Unkwown,0.000000,0.000000e+00,0.000000,0.000000
2132697,ZYHE,VTBT,HEK,127.308884,50.171621,CN,AS,Unknown,100.956802,13.232426,...,NB,B752,183.490826,1,OpenSky,Unkwown,4745.069035,8.706766e+05,717437.547335,21938.233003
2132698,ZYHE,ZGSD,HEK,127.308884,50.171621,CN,AS,ZUH,113.375999,22.006399,...,NB,B738,350.715159,2,OpenSky,CZ,3355.093124,1.176682e+06,969585.984420,11849.285279


In [35]:
os_df["fuel_burn"] = os_df["fuel_burn_seymour"] * os_df["n_flights"]
os_df["co2"] = os_df["fuel_burn"] * 3.16

os_df["domestic"] = (os_df.departure_country == os_df.arrival_country).astype(int)

In [36]:
os_df.to_csv("data/os_alone.csv")

In [37]:
os_df.n_flights.sum()

31218164